In [3]:
import pandas as pd
import glob
import os
import numpy as np
from datetime import datetime
from datetime import date, timedelta
import filepath 
import infrastructure
import pygsheets 
import requests
from requests.structures import CaseInsensitiveDict
from os import listdir
from os.path import isfile, join
import datetime as dt
now = datetime.now()
current_time = now.strftime("%D %H:%M:%S")
current_date = now.strftime("%y%m%d")
print("Last update time:", current_time)

Last update time: 09/06/23 19:34:04


In [4]:
offer_sms = infrastructure.get_smartsheet('offers_sms')
test_sms = infrastructure.get_smartsheet('test_sms')

In [3]:
## Enter the data foler you place output the data file 
output_filepath = filepath.output_folder
## Enter the data foler you place store the input data 
input_filepath = filepath.input_folder
Segment_filepath = filepath.output_folder+'Segment/'
smsreport_filepath = filepath.output_folder+'SMS_Report/'
smsapikey_filepath = filepath.output_folder+'SMS_Apikey/'

In [4]:

offer_sms1 = offer_sms[offer_sms['Hitpath ID'].isna()== False]
#offer_sms1['LC Offer'] =  offer_sms1['Hitpath ID'].astype('str').str.split(".",expand = True)[0] + '_'+offer_sms1['Abbreviation'].str.split('_',expand = True)[1] 
#offer_sms1 = offer_sms1[offer_sms1['LC Offer'].isna()== False]

In [5]:
#offer_sms1['LC Offer'].to_csv('LC_offer.csv')

# GET User data 

In [6]:
"""
# If you want to get the jump page click data from a specific date range 
start_date = '2023-05-01'
end_date = '2023-08-22'
# start_date  
date_range = pd.date_range(start=start_date, end=end_date)

url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=person&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"
resp = requests.get(url)
with open(output_filepath + 'User_information'+current_date+'.csv', 'w+') as f:
    f.write(resp.text)
# The code I downlodaed in the trinity, I want to combine it all in one files 
## Get the jump page clicks 

 """

'\n# If you want to get the jump page click data from a specific date range \nstart_date = \'2023-05-01\'\nend_date = \'2023-08-22\'\n# start_date  \ndate_range = pd.date_range(start=start_date, end=end_date)\n\nurl = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=person&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"\nresp = requests.get(url)\nwith open(output_filepath + \'User_information\'+current_date+\'.csv\', \'w+\') as f:\n    f.write(resp.text)\n# The code I downlodaed in the trinity, I want to combine it all in one files \n## Get the jump page clicks \n\n '

## Get Jump Page Clicks Number

In [7]:
#import new mamba 
# get active pubid 
mamba = infrastructure.get_mamba()
mamba['PUBID'] = mamba['Dataset'].str.split('_',expand = True)[2]
active_pubid =','.join(mamba['PUBID'].unique().tolist())
# yesterday
yesterday = date.today() - timedelta(days=1)
first_date_of_month = yesterday.replace(day=1)
yesterday = yesterday.strftime('%D')
first_date_of_month_url = first_date_of_month.strftime('%m/%d/%Y')
current_date =  first_date_of_month.strftime("%y%m")
url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"
resp = requests.get(url)
with open(output_filepath + 'Jump Page Click from API/jumppage_click'+current_date+'.csv', 'w+') as f:
    f.write(resp.text)
""" 
# If you want to get the jump page click data from a specific date range 
start_date = '2023-05-25'
end_date = '2023-05-31'
# start_date  
date_range = pd.date_range(start=start_date, end=end_date)
for date in date_range:
    current_date_url = date.strftime('%m/%d/%Y')
    current_date = date.strftime("%y%m%d")
    url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={current_date_url}&end={current_date_url}&agents={active_pubid}"
    resp = requests.get(url)
    with open(output_filepath + 'Jump Page Click from API/jumppage_click'+current_date+'.csv', 'w+') as f:
        f.write(resp.text)
# The code I downlodaed in the trinity, I want to combine it all in one files 
## Get the jump page clicks 
os.chdir(output_filepath+'Jump Page Click from trinity')

all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
combined_jump_page_csv = pd.concat([pd.read_csv(f) for f in all_files ])
combined_jump_page_csv = combined_jump_page_csv[['date','subid_1','campaign_id','affiliate_id','hit_id']]
combined_jump_page_csv.columns = [['date','subid_1','campaign_id','affiliate_id','hitid']]
####Exporting master revenue file####
combined_jump_page_csv.to_csv(output_filepath+"jumppage_click202211-20230516.csv", index=False, encoding='utf-8-sig')

"""

os.chdir(output_filepath+'Jump Page Click from API')

all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
api_jump = pd.concat([pd.read_csv(f) for f in all_files ])
api_jump = api_jump.rename(columns = {'c1': 'subid_1','campaignid': 'campaign_id'})
api_jump['affiliate_id']= api_jump['subid_1'].astype(str).str.extract(r'(46\d{4})')
api_jump = api_jump[['date','subid_1','campaign_id','affiliate_id','hitid']]
####Exporting master revenue file####
jump_page_csv = pd.read_csv(output_filepath+"jumppage_click202211-20230516.csv")
jump_page = pd.concat([jump_page_csv,api_jump])
jump_page_clicks = jump_page.groupby(['affiliate_id','campaign_id','subid_1'])['hitid'].nunique().reset_index(name = 'Jump Page Clicks')
affiliate_id = jump_page_clicks['affiliate_id'].astype(str).unique().tolist()
jump_page_clicks =jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)]
jump_page_clicks['date'] = jump_page_clicks['subid_1'].str.extract(r'(\d{1,2}[A-Za-z]{3}\d{2})')
# convert the dateinfo to datetime format using pd.to_datetime()
jump_page_clicks['date'] = pd.to_datetime(jump_page_clicks['date'], format='%d%b%y') 
jump_page_clicks.to_csv(output_filepath+ "jumppage_click.csv", index=False, encoding='utf-8-sig')
duplicateRows = jump_page[jump_page['hitid'].duplicated()]
duplicateRows
# if there's duplicates rows, please check! 

/Users/liliguo/Desktop/SMS-NewData/infrastructure.py:60: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5196/2479119519.py:47: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  api_jump = pd.concat([pd.read_csv(f) for f in all_files ])
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5196/2479119519.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  jump_page_clicks =jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)]


,date,subid_1,campaign_id,affiliate_id,hitid
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
101,NaN,NaN,NaN,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN
103,NaN,NaN,NaN,NaN,NaN
104,NaN,NaN,NaN,NaN,NaN


In [8]:
jump_page_clicks['Date'] = jump_page_clicks['subid_1'].str.extract(r'(\d{1,2}[A-Za-z]{3}\d{2})')
# convert the dateinfo to datetime format using pd.to_datetime()
jump_page_clicks['Date'] = pd.to_datetime(jump_page_clicks['Date'], format='%d%b%y') 
jump_page_clicks

,affiliate_id,campaign_id,subid_1,Jump Page Clicks,date,Date
6,460654,4851.0,MP_HZB_460654_CT4713_I.RC_C15_6264_44345_18Aug...,1,2021-08-18,2021-08-18
7,460654,4851.0,SS_SC_HZB_44345_460654_418966_6264_146928_8Nov...,3,2022-11-08,2022-11-08
8,460654,4851.0,SS_SC_HZB_44345_460654_418966_6264_295469_8Nov...,7,2022-11-08,2022-11-08
9,460654,4851.0,SS_SC_HZB_44345_460654_420034_4851_296307_9Nov...,1,2022-11-09,2022-11-09
10,460654,4851.0,SS_SC_HZB_44345_460654_420034_4851_296307_9Nov...,1,2022-11-09,2022-11-09
...,...,...,...,...,...,...
2024175,461768,12088.0,SS_SC_HZB_44345_461768_0_AR1_12088 _514532_2Sep23,1,2023-09-02,2023-09-02
2024176,461768,12088.0,SS_SC_HZB_44345_461768_591329_12088_515339_30A...,13,2023-08-30,2023-08-30
2024177,461768,12088.0,SS_SC_MBC_80837_461768_591672_12088_516084_31A...,1,2023-08-31,2023-08-31
2024178,461768,12088.0,SS_SC_MBC_80837_461768_591672_12088_516085_31A...,5,2023-08-31,2023-08-31


In [9]:

useful_click = jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)]['Jump Page Clicks'].sum()
useless_clicks = jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False) == False]['Jump Page Clicks'].sum()
useful_click+useless_clicks -  len(jump_page) 
# the missing 502 clicks is from problematic subid_1 
# useless clicks is also from porblematic clicks 

/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5196/322886987.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  useful_click = jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)]['Jump Page Clicks'].sum()
/var/folders/81/9wfkcg6j573945pbm52zxh_80000gn/T/ipykernel_5196/322886987.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  useless_clicks = jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False) == False]['Jump Page Clicks'].sum()


-2397

## Update Flow

In [10]:
legacy_flows = pd.read_csv(output_filepath+'SMS_SC_SS_Flows.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_flows.to_csv(input_filepath+'SMS_SC_SS_Flows_backup.csv',index=False)
# This is the creative stat raw data downloaded by the outsource 
updated_flows = pd.read_csv(input_filepath+'SMS_SC_SS_Flows.csv',index_col=False)
legacy_flows['Period'] = pd.to_datetime(legacy_flows['Period'],format='mixed')
min_date = updated_flows['Period'].min()
legacy_flows = legacy_flows[legacy_flows['Period']<min_date]
flows = [legacy_flows, updated_flows]
output_flows = pd.concat(flows, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_flows)
output_flows.to_csv(output_filepath+'SMS_SC_SS_Flows.csv', index = False)


         Id                   Name               Period  \
0      5009          PA.SWP-MBC-PR  2023-03-06 00:00:00   
1      5009          PA.SWP-MBC-PR  2023-03-07 00:00:00   
2      5009          PA.SWP-MBC-PR  2023-03-07 00:00:00   
3      5009          PA.SWP-MBC-PR  2023-03-08 00:00:00   
4      5009          PA.SWP-MBC-PR  2023-03-08 00:00:00   
...     ...                    ...                  ...   
12982  5104      RHD.CC-HZB-PR-SFW           2023-09-05   
12983  5104      RHD.CC-HZB-PR-SFW           2023-09-05   
12984  5105  RHD.CC-DSS-TMO-BU-SFW           2023-09-05   
12985  5105  RHD.CC-DSS-TMO-BU-SFW           2023-09-05   
12986  5105  RHD.CC-DSS-TMO-BU-SFW           2023-09-05   

                                          Shortcode                 ListName  \
0      MyBenefitsClub :: MBC :: 80837 [80837 : 306]   MyBenefitsClub [12421]   
1      MyBenefitsClub :: MBC :: 80837 [80837 : 306]   MyBenefitsClub [12421]   
2      MyBenefitsClub :: MBC :: 80837 [80837 : 306]

## Update Creatives Data

In [11]:
legacy_creatives = pd.read_csv(output_filepath+'SMS_SC_SS_CreativesStats.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_creatives.to_csv(input_filepath+'SMS_SC_SS_CreativesStats_backup.csv',index=False)
# This is the creative stat raw data downloaded by the outsource 
updated_creatives = pd.read_csv(input_filepath+'creativestats.csv',index_col=False)
legacy_creatives['Tstamp'] = pd.to_datetime(legacy_creatives['Tstamp'],format='mixed')
min_date = updated_creatives['Tstamp'].min()
legacy_creatives = legacy_creatives[legacy_creatives['Tstamp']<min_date]
creativestats = [legacy_creatives, updated_creatives]
output_creatives = pd.concat(creativestats, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_creatives)
output_creatives['Tstamp'] = pd.to_datetime(output_creatives['Tstamp'],format='mixed')
output_creatives.to_csv(output_filepath+'SMS_SC_SS_CreativesStats.csv', index = False)


        JobId                     Tstamp                              Offer  \
0           0  2021-10-01 00:00:00+00:00                                NaN   
1      237891  2021-10-01 17:00:00+00:00  B2USRA - Cheris Studio Benefits A   
2           0  2021-10-02 00:00:00+00:00                                NaN   
3      238383  2021-10-02 17:00:00+00:00  B2USRA - Cheris Studio Benefits A   
4           0  2021-10-03 00:00:00+00:00                                NaN   
...       ...                        ...                                ...   
31105  592990       2023-09-05T17:00:55Z                 6322 - BB VAST MBC   
31106  592966       2023-09-05T17:00:56Z                 6322 - BB VAST MBC   
31107  592968       2023-09-05T17:00:57Z                   5871 - NC BD MBC   
31108  593081       2023-09-05T19:00:01Z                  7878 - DA ADT DSS   
31109  593081       2023-09-05T19:00:01Z                  7878 - DA ADT DSS   

                              Segment  CreativeId  

In [12]:
legacy_jobs = pd.read_csv(output_filepath+'SMS_SC_SS_Jobs.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_jobs.to_csv(input_filepath+'SMS_SC_SS_Jobs_backup.csv', index = False)
# This is the job stat raw data downloaded by the outsource 
updated_jobs = pd.read_csv(input_filepath+'jobs.csv',index_col=False)
legacy_jobs['scheduled_tstamp'] = pd.to_datetime(legacy_jobs['scheduled_tstamp'],format = "mixed")
min_date = updated_jobs['scheduled_tstamp'].min()
legacy_jobs = legacy_jobs[legacy_jobs['scheduled_tstamp']<min_date]
jobs = [legacy_jobs, updated_jobs]
output_jobs = pd.concat(jobs, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_jobs)
output_jobs.to_csv(output_filepath+'SMS_SC_SS_Jobs.csv', index = False)

           id                                    name  \
0      237891                   B2USRA_1 [2021-10-01]   
1      238383                   B2USRA_1 [2021-10-02]   
2      238832                   B2USRA_1 [2021-10-03]   
3      239249                   B2USRA_1 [2021-10-04]   
4      239695                   B2USRA_1 [2021-10-05]   
...       ...                                     ...   
24838  593036       SS_MBC_PN-FC-15DC_11468_P_05Sep23   
24839  593042      SS_MBC_PN-FC-15DC_12363_PT_05Sep23   
24840  593054   SS_FLC_PN-FC-15DC-VZN_11646_P_05Sep23   
24841  593062  SS_FLC_PN-FC-15DC-VZN_12364_PT_05Sep23   
24842  593081       SS_DSS_TLG-PL-30DC_7878_P_05Sep23   

                                   offer  shortcode             start_tstamp  \
0      B2USRA - Cheris Studio Benefits A      70610  2021-10-01 13:00:09 EST   
1      B2USRA - Cheris Studio Benefits A      70610  2021-10-02 13:00:10 EST   
2      B2USRA - Cheris Studio Benefits A      70610  2021-10-03 13:00:16 ES

## Update Jobs

In [13]:
offers_old = pd.read_csv(output_filepath + 'SMS_SC_SS_Offers.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong
offers_old.to_csv(input_filepath+'SMS_SC_SS_Offers_backup.csv', index = False)
# This is the offer raw data downloaded by the outsource 
offers = pd.read_csv(input_filepath+'SMS_SC_SS_Offers.csv',index_col=False)
offers['hitpath_offer_id'] = offers['name'].str.split(' ',expand = True)[0].str.strip()
offers = offers[['id', 'name','hitpath_offer_id', 'type', 'status', 'redirect_type', 'conversion_event',
       'conversion_payout', 'currency']]
print(offers.head(5))
offers.to_csv(output_filepath+'SMS_SC_SS_Offers.csv', index = False)

      id                  name hitpath_offer_id type  status redirect_type  \
0  28905     10035 - DB ML CSS            10035  sms  active          HTTP   
1  27287     10035 - DB ML HZB            10035  sms  active          HTTP   
2  27665     10086 - BB NP CSS            10086  sms  active          HTTP   
3  27676     10086 - BB NP HZB            10086  sms  active          HTTP   
4  30545  10232 - RX RTOCS CSS            10232  sms  active          HTTP   

  conversion_event  conversion_payout currency  
0              CPA                  0      USD  
1              CPA                  0      USD  
2              CPA                  0      USD  
3              CPA                  0      USD  
4              CPA                  0      USD  


## Update Segment

In [14]:

from os import listdir
from os.path import isfile, join
import datetime as dt

mamba = infrastructure.get_mamba()
current_active_segment = mamba.loc[mamba['Date'] == pd.to_datetime(date.today()),'Segment '].unique().tolist()
secondary_segment =  mamba.loc[(mamba['Date'] == pd.to_datetime(date.today())) & (mamba['Segment '].str.contains('TMO|VZN',na = False)),'Segment '].unique().tolist()

                                                                 
segment_files = [f for f in listdir(Segment_filepath) if isfile(join(Segment_filepath, f)) & ('SMS_SC_SS_Segments' in f)]
output_segment = pd.DataFrame()
segment_files.sort()
for i in segment_files: 
    segment_report = pd.read_csv(Segment_filepath+i)
    segment_report['Date'] = dt.datetime.strptime((i.split('.csv')[0][-8:]), "%Y%m%d")
    updated_segment_report = segment_report.drop(['id','next_count'],axis = 1)
    output_segment = pd.concat([updated_segment_report,output_segment], axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
    print(output_segment)
""" 
most_recent_segment_date = max([dt.datetime.strptime((date.split('.csv')[0][-8:]), "%Y%m%d") for date in segment_files])
segment_year = most_recent_segment_date.year
segment_month = most_recent_segment_date.month
segement_day = most_recent_segment_date.day

most_recent_segment_file = "SMS_SC_SS_Segments{0}{1:0>2}{2:0>2}.csv".format(segment_year,segment_month,segement_day)

segment_report = pd.read_csv( Segment_filepath+ most_recent_segment_file)

segment_report['Date'] = pd.to_datetime(dt.datetime.today().strftime("%D"))
updated_segment_report = segment_report.drop(['id','next_count'],axis = 1)

legacy_segment = pd.read_csv(output_filepath+'SMS_SC_SS_Segments.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_segment.to_csv(input_filepath+'SMS_SC_SS_Segments_backup.csv', index = False)
# This is the job stat raw data downloaded by the outsource 
legacy_segment['Date'] = pd.to_datetime(legacy_segment['Date'])
min_date = updated_segment_report['Date'].min()
legacy_segment = legacy_segment[legacy_segment['Date']<min_date]
full_segment = [legacy_segment, updated_segment_report]
output_segment = pd.concat(full_segment, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_segment)
"""
## rename on 5/24
output_segment['name'] = output_segment['name'].str.replace('FLC_I.CC_OI.4P_30DC','FLC_I.CC_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_SPK.CR_OI.4P_30DC','FLC_SPK.CR_30DC')
output_segment['name'] = output_segment['name'].str.replace('MBC_SPK.SWP_OI.4P_30DC','MBC_SPK.SWP_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_AI.CC_OI.4P_30DC','FLC_AI.CC_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_EDM.247L_OI.4P_30DC','FLC_EDM.247L_30DC')
output_segment['name'] = output_segment['name'].str.replace('MBC_NPD.RTO_OI.4P_30DC','MBC_NPD.RTO_30DC')
output_segment['name'] = output_segment['name'].str.replace('MBC_PN.FC_OI.4P_30DC','MBC_PN.FC_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_WWM.YFA.2_OI.4P_30DC','FLC_WWM.YFA.2_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_ZM.PL.2_OI.4P_30DC','FLC_ZM.PL.2_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_SPK.SWP_OI.4P.VZN_30DC','FLC_SPK.SWP_30DC_VZN')
output_segment['name'] = output_segment['name'].str.replace('MBC_EDM.247L_OI.4P.TMO_30DC','MBC_EDM.247L_30DC_TMO')
output_segment['name'] = output_segment['name'].str.replace('MBC_WWM.YFA.2_OI.4P_30DC','MBC_WWM.YFA.2_30DC_TMO')
output_segment['Secondary Segment'] = False
output_segment['Active Segment'] = False
output_segment.loc[output_segment['name'].isin(secondary_segment),'Secondary Segment'] = True
output_segment.loc[output_segment['name'].isin(current_active_segment),'Active Segment'] = True

output_segment.to_csv(output_filepath+'SMS_SC_SS_Segments.csv', index = False)

/Users/liliguo/Desktop/SMS-NewData/infrastructure.py:60: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')


                             name channel  status  count  \
0                          461243     sms  active  11531   
1          526 rto-JULY22-CLK-hzb     sms  active   1283   
2                   ARM CR 460939     sms  active  62935   
3               ARM.CR Last Month     sms  active  26541   
4                   ARMSMPL461149     sms  active   8279   
..                            ...     ...     ...    ...   
304                RTO-JULY22-CLK     sms  active   7579   
305               test-2565371505     sms  active   4255   
306  viewrenttoownhomes URL check     sms  active   6386   
307                      ZP461235     sms  active  85116   
308                 ZP Last Month     sms  active   4110   

                last_counted       Date  
0    2023-05-01 00:06:08 EST 2023-05-17  
1    2023-05-17 06:18:34 EST 2023-05-17  
2    2023-05-01 19:42:33 EST 2023-05-17  
3    2023-04-29 06:55:26 EST 2023-05-17  
4    2023-04-29 04:34:56 EST 2023-05-17  
..                       ..

## Update Post Data

In [15]:
# Output dailt post data
legacy_post = pd.read_csv(output_filepath+'SMSposts.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_post.to_csv(input_filepath+'SMSposts_backup.csv',index=False)
# This is the creative stat raw data downloaded by the outsource 
updated_post = pd.read_csv(input_filepath+'SMSPosts.csv',index_col=False)
drop = updated_post['Pub Name'].str.contains('SMS', case = False, na = False)
updated_post = updated_post.loc[drop]
today = date.today()
first_date_of_month = today.replace(day=1)
updated_post = updated_post.loc[updated_post['Pub Id'].isna() == False]
current_year = datetime.now().year
updated_post['Dates'] = pd.to_datetime(updated_post['Dates'], format="%m/%d").apply(lambda x: x.replace(year=current_year))
updated_post['Dates'] = pd.to_datetime(updated_post['Dates']).dt.strftime('%m/%d/%Y')
updated_post = updated_post[updated_post['Dates']<=yesterday]
#updated_post['Dates'] = pd.to_datetime(updated_post['Dates']).dt.strftime('%m/%d/%Y')
legacy_post['Dates'] = pd.to_datetime(legacy_post['Dates'] )
min_date = updated_post['Dates'].min()
legacy_post = legacy_post[legacy_post['Dates']<min_date]
legacy_post['Dates'] = pd.to_datetime(legacy_post['Dates']).dt.strftime('%m/%d/%Y')
post = [legacy_post, updated_post]
output_post= pd.concat(post, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_post)
output_post['Pub Id'] = output_post['Pub Id'].astype(str)
output_post['Pub Name'] = output_post['Pub Name'].astype(str)
output_post = output_post.loc[output_post['Pub Name'].str.contains('SMS', case = False, na = False)]
output_post['Dates'] =  pd.to_datetime(output_post['Dates']) 
output_post.to_csv(output_filepath+'SMSposts.csv', index = False)

            Dates    Pub Id                           Pub Name  Raw Posts  \
0      01/01/2022  461227.0       LM_SMS_Edemographic_247Loans          0   
1      01/01/2022  461313.0  LM_SMS_NorthPointDirect_RentToOwn          0   
2      01/01/2022  460921.0            LM_SMS_Spark_CreditCard      17102   
3      01/01/2022  461147.0                 461147_ARM.SYF_SMS      15884   
4      01/01/2022  460917.0                   460917_JM.OP_SMS       9999   
...           ...       ...                                ...        ...   
29159  09/05/2023  461228.0                  461228_ED.PFR_SMS          0   
29160  09/05/2023  461148.0                  461148_ARM.CF_SMS          0   
29161  09/05/2023  461420.0                1 - SMS LM : BH.ROE          0   
29162  09/05/2023  461110.0                 I_RxMG_SMS Actives          0   
29163  09/05/2023  461348.0                   461348_I.CHQ_SMS          0   

       GeoIP Good  % IP Good  GeoIP Bad  % IP Bad  Goodhyg  % Good  ...  \


## Update Optout 

In [16]:
from os import listdir
from os.path import isfile, join
import datetime as dt
                                                                     
smsreport_files = [f for f in listdir(smsreport_filepath) if isfile(join(smsreport_filepath, f)) & ('SMS_SC_SS_Report' in f)]
""" 
most_recent_smsreport_date = max([dt.datetime.strptime((date.split('.csv')[0][-8:]), "%Y%m%d") for date in smsreport_files])
smsreport_year = most_recent_smsreport_date.year
smsreport_month = most_recent_smsreport_date.month
segement_day = most_recent_smsreport_date.day

most_recent_smsreport_file = "SMS_SC_SS_Report{0}{1:0>2}{2:0>2}.csv".format(smsreport_year,smsreport_month,segement_day)

smsreport_report = pd.read_csv( smsreport_filepath+ most_recent_smsreport_file)

smsreport_report['Date'] = pd.to_datetime(most_recent_smsreport_date)
smsreport_report['Shortcode Name'] = ''
smsreport_report.loc[smsreport_report['Shortcode'].str.contains('FLC'),'Shortcode Name' ] ='FLC'
smsreport_report.loc[smsreport_report['Shortcode'].str.contains('HZB'),'Shortcode Name' ] ='HZB'
smsreport_report.loc[smsreport_report['Shortcode'].str.contains('MBC'),'Shortcode Name' ] ='MBC'
smsreport_report.loc[smsreport_report['Shortcode'].str.contains('CSS'),'Shortcode Name' ] ='CSS'
legacy_smsreport = pd.read_csv(output_filepath+'SMS_SC_SS_Report.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_smsreport.to_csv(input_filepath+'SMS_SC_SS_Report_backup.csv', index = False)
# This is the job stat raw data downloaded by the outsource 
legacy_smsreport['Date'] = pd.to_datetime(legacy_smsreport['Date'])
min_date = smsreport_report['Date'].min()
legacy_smsreport = legacy_smsreport[legacy_smsreport['Date']<min_date]
full_smsreport = [legacy_smsreport, smsreport_report]
output_smsreport = pd.concat(full_smsreport, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_smsreport)
output_smsreport.to_csv(output_filepath+'SMS_SC_SS_Report.csv', index = False)
"""
df = pd.DataFrame()
for i in smsreport_files:
    smsreport_report = pd.read_csv(smsreport_filepath+i)
    most_recent_smsreport_date = dt.datetime.strptime((i.split('.csv')[0][-8:]), "%Y%m%d")
    smsreport_report['Date'] = pd.to_datetime(most_recent_smsreport_date)
    smsreport_report['Shortcode Name'] = ''
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('FLC'),'Shortcode Name' ] ='FLC'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('HZB'),'Shortcode Name' ] ='HZB'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('MBC'),'Shortcode Name' ] ='MBC'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('CSS'),'Shortcode Name' ] ='CSS'
    df = pd.concat([df,smsreport_report])
df.to_csv(output_filepath+'SMS_SC_SS_Report.csv', index = False)


# update apikey

In [17]:
from os import listdir
from os.path import isfile, join
import datetime as dt
                                                                     
smsreport_files = [f for f in listdir(smsapikey_filepath) if isfile(join(smsapikey_filepath, f)) & ('apikeyreport' in f)]

df = pd.DataFrame()
for i in smsreport_files:
    smsapikey_report = pd.read_csv(smsapikey_filepath+i)
    smsapikey_report['Date'] = pd.to_datetime(smsapikey_report['Date'])
    smsapikey_report['DP.SV'] = smsapikey_report['Apikey'].str.split(" |-",expand = True)[0]
    df = pd.concat([df,smsapikey_report])
df = df.drop_duplicates()
df.to_csv(output_filepath+'SMS_SC_SS_Apikey.csv', index = False)
